In [1]:
from my_parser import *

In [2]:
import re
m = re.match(r'a{3}','aaaaaa')
m.group(0)

'aaa'

In [13]:
depth = 0
def cache_debug(name):
    def decorator(fun):
        def wrapper(s,p):
            global depth
            #print('  '*depth,name,p)
            depth+=1
            try:
                rezs=fun(s,p)   # <<<<<================== CALL FUN ======================
            finally:
                depth-=1
            return rezs
        return wrapper
    return decorator

In [4]:
@method(Parser)
def add_regexp(self,name,patt):
    cp = re.compile(patt)
    def pp_regexp(s,p):
        m = cp.match(s[p:])
        if m==None:
            return Elist(p,FatalError('ожидалось рег.выр. '+name))
        else:
            return [(p+len(m.group(0)),m.group(0))]
    self.patterns[name] = cache_debug(name)(pp_regexp)

In [5]:
@method(Parser)
def add_reg_seq(self,name,patt):
    p,r = self.p_reg_sequence(patt,0)
    if is_fatal(r):
        raise BaseException(r)
    tmp = r
    p,r = p_spcs(patt,p)
    if p!=len(patt):
        raise BaseException('разобран не весь паттерн')
    self.patterns[name] = cache_debug(name)(tmp[1])


In [6]:
@method(Parser)
def add_bnf_seq(self,name,patt):
    p,r = self.p_bnf_sequence_(patt,0)
    if is_fatal(r):
        raise BaseException(r)
    tmp = r
    if p!=len(patt):
        raise BaseException('разобран не весь паттерн')
    self.patterns[name] = cache_debug(name)(tmp[1])
    

In [7]:
@method(Parser)
def add_reg_alt(self,name,patt):
    p,r = self.p_reg_alternatives(patt,0)
    if is_fatal(r):
        raise BaseException(r)
    tmp = r
    p,r = p_spcs(patt,p)
    if p!=len(patt):
        raise BaseException('разобран не весь паттерн')
    self.patterns[name] = cache_debug(name)(tmp[1])


In [8]:
@method(Parser)
def add_bnf_alt(self,name,patt):
    p,r = self.p_bnf_alternatives_(patt,0)
    if is_fatal(r):
        raise BaseException((p,r))
    tmp = r
    if p!=len(patt):
        raise BaseException('разобран не весь паттерн')
    self.patterns[name] = cache_debug(name)(tmp[1])


In [15]:
calculator = Parser()
calculator.add_bnf_seq('num','?=[0-9]+/*s:int(s)*/')
calculator.add_bnf_alt('rterm','$(num)|`(`$(sum)`)`')
calculator.add_bnf_alt('term',
'''$(rterm) 
|?= ${t:rterm} `*` ${n:term} /* t,n:t*n */ 
|?= ${t:rterm} `/` ${n:term} /* t,n:t/n */ 
''')
calculator.add_bnf_alt('sum',
'''$(term)
|?= ${t:term} `+` ${n:sum} /* t,n:t+n */ 
|?= ${t:term} `-` ${n:sum} /* t,n:t-n */ 
''')
calculator.main = 'sum'

In [16]:
calculator.parse('2+2*2')

6

In [11]:
calculator.p_bnf_sequence_('(?t=term) `*` (?n=rterm) /* t,n:t*n */',0)

у неявной объектной последовательности обработчики НЕ допустимы


(38,
 FatalError('у неявной объектной последовательности обработчики НЕ допустимы'))

In [12]:
calculator.p_bnf_sequence_('?= (?t=term) `*` (?n=rterm) /* t,n:t*n */',0)

(41,
 ('direct',
  <function my_parser.make_sequence.<locals>.pp_direct_obj_join(s, p0)>))